# Fusionner les lexèmes de plusieurs kalabas

- par exemple pour Mahira et Agathos

In [17]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle
#from cellbell import ding

def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

In [18]:
%store -r numerosKalaba
numeroKalaba="21-K%d"%numerosKalaba[0]
print numeroKalaba

21-K5


## Ouvrir les deux lexiques

In [19]:
home = expanduser("~")
serieTarget=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s/"%"21-K5"
serieSource=home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s/"%"21-K4"

with open(serieTarget+"Stems.yaml", 'r') as stream:
    lexTarget=yaml.safe_load(stream)
with open(serieSource+"Stems.yaml", 'r') as stream:
    lexSource=yaml.safe_load(stream)

In [20]:
lexTarget

{'ADJ': {'A1': {'Spuk': ['six'],
   'bekut': ['nombreux', 'nombreuses'],
   'bugoN': [u'h\xe9ro\xefque', u'h\xe9ro\xefques'],
   'kink': ['cinq'],
   'loNak': ['deux'],
   'poker': ['trois'],
   'pujol': ['sept'],
   'runuk': ['noir', 'noire', 'noires', 'noirs'],
   'ruwor': ['dernier', u'derni\xe8re', 'derniers', u'derni\xe8res']},
  'A2': {'Tonud': ['vert', 'verte', 'vertes', 'verts'],
   'bewep': ['disparu', 'disparues', 'disparue', 'disparus'],
   'bunig': ['autre', 'autres'],
   'dikeT': ['nouveau', 'nouvelle', 'nouvelles', 'nouveaux'],
   'gepaT': [u'm\xe9chant', u'm\xe9chante', u'm\xe9chantes', u'm\xe9chants'],
   'manaN': ['quatre'],
   'okarina': [u'troisi\xe8me', u'troisi\xe8mes'],
   'savat': ['grand', 'grands', 'grande', 'grandes']}},
 'DET': {'b': ['DEM', 'ce', 'cet', 'cette', 'ces'],
  'k': ['DEF', 'le', 'la', 'les', 'l'],
  'l': ['IND', 'un', 'une', 'des', 'dE']},
 'NOM': {'N1': {'A': {'DuguN': [u'for\xeat', u'for\xeats'],
    'Nipet': ['bras', 'braS'],
    'dimuN': ['ho

## Générer des radicaux nouveaux
- sur la base de la fréquence des consonnes
- qui ne soient pas dans refStems

In [21]:
from numpy.random import choice

consonnesFreq=[10,10,10,10,10,10,8,8,6,7,7,7,7,5,5,5,5,8,8,3,3]
consonnesTot=sum(consonnesFreq)
consonnesProb=[float(k)/consonnesTot for k in consonnesFreq]


with open(serieTarget+"Phonology.yaml", 'r') as stream:
    phonology=yaml.safe_load(stream)


consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]

In [22]:
def randomCVCVC(refStems=[]):
    result=""
    result+=choice(list(consonnes), p=consonnesProb)
    result+=rd.choice(voyelles)
    result+=choice(list(consonnes), p=consonnesProb)
    result+=rd.choice(voyelles)
    result+=choice(list(consonnes), p=consonnesProb)
    if result not in refStems:
        return result
    else:
        return randomCVCVC(refStems)

In [23]:
randomCVCVC()

'betiT'

## Récuperer les informations lexicales

In [24]:
def getLexemes(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element]) is list:
#             formesCle=", ".join()
            formes[element]=stemStructure[element]
        else:
            formes.update(getLexemes(stemStructure[element]))
    return formes

In [25]:
def getLemmes(stemStructure):
#     print stemStructure
    lexemes=getLexemes(stemStructure)
    result=[v[0] for k,v in lexemes.items()]
    return result

In [26]:
def getFrench(stemStructure):
    lexemes=getLexemes(stemStructure)
    result=[v for k,v in lexemes.items()]
    return result

In [27]:
def getStems(stemStructure):
    lexemes=getLexemes(stemStructure)
    result=[k for k,v in lexemes.items()]
    return result    

In [28]:
stemsSource=getStems(lexSource)
stemsTarget=getStems(lexTarget)

## Ajout des lexèmes manquants

In [29]:
for cat in lexSource:
    if cat!="PRO":
        frSource=getFrench(lexSource[cat])
        missingLexemes=set(getLemmes(lexSource[cat]))-set(getLemmes(lexTarget[cat]))

        stemsTargetNew=stemsTarget[:]
        for l in frSource:
            if l[0] in missingLexemes:
                print l
                radical=randomCVCVC(stemsTargetNew)
                stemsTargetNew.append(radical)
                lexTarget[cat][radical]=l
        
# lexAgathos    

['chatte', 'chattes']
['chef', 'chefs', 'cheffe', 'cheffes']
['lance', 'lances']
['nouvelle-NOM', 'nouvELLE', 'nouvELLEs']
['cuisine', 'cuisines']
['couteau', 'couteaux']
['viande', 'viandes']
['loup', 'loups']
['visage', 'visages']
['arbre', 'arbres']
['Kaleb', 'Kalebs']
['Violette', 'Violettes']
[u'col\xe8re', u'col\xe8res']
['ombre', 'ombres']
['chambre', 'chambres']
['coyote', 'coyotes']
[u'd\xe9mon', u'd\xe9mons']
['main', 'mains']
['Mahira', 'Mahiras']
['lit', 'lits']
['blessure', 'blessures']
['Nicole', 'Nicoles']
['Katisha', 'Katishas']
['souffrance', 'souffrances']
['lune', 'lunes']
['Nabil', 'Nabils']
['sort', 'sorts']
[u'th\xe9', u'th\xe9s']
[u'caf\xe9', u'caf\xe9s']
[u'\u0153uf', 'oeuf', u'\u0153ufs', 'oeufs']
['louve', 'louves', 'lOUP', 'lOUPs']
['coussin', 'coussins']
['poisson', 'poissons']
['serpent', 'serpents']
[u'infirmi\xe8re', u'infirmi\xe8res']
['maison', 'maisons']
['lueur', 'lueurs']
['dragon', 'dragons']
['table', 'tables']
['fruit', 'fruits']
['coup', 'coups']

## Stats sur les phonèmes dans le lexique

In [30]:
def statsCV(lstStems):
    cons={}
    vocs={v:0 for v in "aeiou"}
    for s in lstStems:
        for c in s:
            if c not in vocs: 
                if c not in cons: cons[c]=0
                cons[c]+=1
            else:
                vocs[c]+=1
    return cons,vocs

In [31]:
stems=getStems(lexSource)
statsCV(stems)

({'D': 24,
  'N': 26,
  'S': 30,
  'T': 36,
  'Z': 35,
  'b': 56,
  'd': 39,
  'f': 37,
  'g': 48,
  'j': 17,
  'k': 63,
  'l': 48,
  'm': 45,
  'n': 42,
  'p': 43,
  'r': 50,
  's': 32,
  't': 65,
  'v': 28,
  'w': 11,
  'z': 14},
 {'a': 131, 'e': 101, 'i': 92, 'o': 104, 'u': 101})

## Sauvegarde du nouveau lexique

In [32]:
with open(serieTarget+'fusedStems.yaml', 'w') as output:
    yaml.dump(lexTarget, output, default_flow_style=False,allow_unicode=True)

In [33]:
with open(serieTarget+"fusedStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [34]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  A1:
    Spuk: [six]
    bekut: [nombreux, nombreuses]
    bugoN: [héroïque, héroïques]
    kink: [cinq]
    loNak: [deux]
    poker: [trois]
    pujol: [sept]
    runuk: [noir, noire, noires, noirs]
    ruwor: [dernier, dernière, derniers, dernières]
  A2:
    Tonud: [vert, verte, vertes, verts]
    bewep: [disparu, disparues, disparue, disparus]
    bunig: [autre, autres]
    dikeT: [nouveau, nouvelle, nouvelles, nouveaux]
    gepaT: [méchant, méchante, méchantes, méchants]
    manaN: [quatre]
    okarina: [troisième, troisièmes]
    savat: [grand, grands, grande, grandes]
  Nipik: [petit, petits, petite, petites]
  NuNef: [rouge, rouges]
  SakaT: [jaune, jaunes]
  Tofed: [blessé, blessés, blessée, blessées]
  Zoset: [profond, profonds, profonde, profondes]
  bupid: [bas, basse, basses]
  dezid: [maigre, maigres]
  gamiz: [terrible, terribles]
  gozek: [gros, grosse, grosses]
  jaNib: [blanc, blancs, blanche, blanches]
  kobaT: [vilain, vilains, vilaine, vilaines]
  lumiD: [tre

In [35]:
with open(serieTarget+'fusedStems.yaml', 'w') as output:
    output.write(yamlText)